In [3]:
import torch
import numpy as np
import pandas as pd
from ecg_bib import ECG_Dataset, ECG_CNN_Classifier, train
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import f1_score
from tqdm import tqdm

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_dataset = ECG_Dataset('../data/train_dccweek2023.h5',
                            '../data/train_dccweek2023-labels.csv',
                             start = 0, end = 45000)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

val_dataset = ECG_Dataset('../data/train_dccweek2023.h5',
                            '../data/train_dccweek2023-labels.csv',
                             start = 45000, end = -1)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

model = ECG_CNN_Classifier().to(device)

criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=0.001)

In [6]:
batch = next(iter(val_loader))

In [18]:
batch[0][6][0]

tensor(0.8842)

In [3]:
train(model, train_loader, val_loader, optimizer, criterion, 20, "models/ECG_CNN_Classifier")


Starting epoch 1 ----------



  6%|██▎                                      | 80/1407 [00:08<02:14,  9.88it/s]


KeyboardInterrupt: 

In [2]:
# treinamento do modelo
for epoch in range(20):
    running_loss = 0.0
    # TREINO
    for i, (inputs, labels) in enumerate(train_loader, 0):
        # separando as entradas e os rótulos
        inputs, labels = inputs.to(device), labels.to(device)
        
        # zerando os gradientes
        optimizer.zero_grad()

        # passando as entradas pelo modelo
        outputs = model(inputs)

        # calculando a perda
        loss = criterion(outputs, labels)

        # realizando o backpropagation e a otimização dos pesos
        loss.backward()
        optimizer.step()

        # printando as estatísticas
        running_loss += loss.item()
        if i % 200 == 199:    # printando a cada 200 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 200))
            running_loss = 0.0
    
    # VALIDAÇÃO
    model.eval()
    val_loss = 0
    val_total = 0
    val_correct = 0
    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            val_loss += loss.item() * labels.size(0)
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_loss = val_loss / len(val_set)
    val_acc  = val_correct / val_total
    print('Epoch [{}/{}], Train Loss: {:.4f}, Train Acc: {:.4f}, Val Loss: {:.4f}, Val Acc: {:.4f}'.format(epoch+1, num_epochs, train_loss, train_acc, val_loss, val_acc))
print('Treinamento finalizado')

# salvando o modelo
PATH = './models/ecg_conv.pth'
torch.save(model.state_dict(), PATH)

[1,   200] loss: 1.425
[1,   400] loss: 0.539
[1,   600] loss: 0.528


KeyboardInterrupt: 